<a href="https://colab.research.google.com/github/sahana1410/samplecode/blob/main/Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# ✅ STEP 1: Install required libraries
!pip install tensorflow opencv-python pillow --quiet


In [2]:
# ✅ STEP 2: Upload Images (style.jpg + content1.jpg to content5.jpg)
from google.colab import files
uploaded = files.upload()

# Your uploaded files are now available in the current working directory (/content/)


Saving content1.jpg to content1.jpg
Saving content2.jpg to content2.jpg
Saving content3.jpg to content3.jpg
Saving content4.jpg to content4.jpg
Saving content5.jpg to content5.jpg
Saving style.jpg to style.jpg


In [3]:
# ✅ STEP 3: Imports and Utility Functions
import tensorflow as tf
import numpy as np
import cv2
import os
from PIL import Image
from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model

# Load and preprocess image
def load_and_process_image(image_path, max_dim=512):
    img = Image.open(image_path)
    img = img.convert("RGB")
    img.thumbnail((max_dim, max_dim))
    img = np.array(img)
    img = np.expand_dims(img, axis=0)
    return vgg19.preprocess_input(img.astype(np.float32))

# Deprocess image
def deprocess_image(x):
    x = x.reshape((x.shape[1], x.shape[2], 3))
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load VGG19 for feature extraction
def get_model():
    vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1',
                    'block4_conv1', 'block5_conv1']
    outputs = [vgg.get_layer(name).output for name in (style_layers + content_layers)]
    return Model(inputs=vgg.input, outputs=outputs), style_layers, content_layers

def gram_matrix(tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', tensor, tensor)
    input_shape = tf.shape(tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result / num_locations

def get_feature_representations(model, content_image, style_image, style_layers, content_layers):
    style_outputs = model(style_image)
    content_outputs = model(content_image)
    style_features = [gram_matrix(style_outputs[i]) for i in range(len(style_layers))]
    content_features = [content_outputs[i + len(style_layers)] for i in range(len(content_layers))]
    return style_features, content_features

def compute_loss(model, loss_weights, init_image, gram_style_features, content_features, style_layers, content_layers):
    style_weight, content_weight = loss_weights
    model_outputs = model(init_image)
    style_output_features = model_outputs[:len(style_layers)]
    content_output_features = model_outputs[len(style_layers):]

    style_score = 0
    content_score = 0

    for target_style, comb_style in zip(gram_style_features, style_output_features):
        style_score += tf.reduce_mean((gram_matrix(comb_style) - target_style) ** 2)

    for target_content, comb_content in zip(content_features, content_output_features):
        content_score += tf.reduce_mean((comb_content - target_content) ** 2)

    style_score *= style_weight
    content_score *= content_weight
    total_loss = style_score + content_score
    return total_loss, style_score, content_score

@tf.function()
def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = compute_loss(**cfg)
    total_loss = all_loss[0]
    return tape.gradient(total_loss, cfg['init_image']), all_loss


In [4]:
# ✅ STEP 4: Style Transfer Function
def run_style_transfer(content_path, style_path, output_path,
                       iterations=300, content_weight=1e3, style_weight=1e-2):
    model, style_layers, content_layers = get_model()

    content_image = load_and_process_image(content_path)
    style_image = load_and_process_image(style_path)

    style_features, content_features = get_feature_representations(
        model, content_image, style_image, style_layers, content_layers)

    init_image = tf.Variable(content_image, dtype=tf.float32)
    opt = tf.optimizers.Adam(learning_rate=5.0)

    best_img = None
    best_loss = float('inf')

    cfg = {
        'model': model,
        'loss_weights': (style_weight, content_weight),
        'init_image': init_image,
        'gram_style_features': style_features,
        'content_features': content_features,
        'style_layers': style_layers,
        'content_layers': content_layers
    }

    for i in range(iterations):
        grads, all_loss = compute_grads(cfg)
        loss, style_score, content_score = all_loss
        opt.apply_gradients([(grads, init_image)])

        clipped = tf.clip_by_value(init_image, -103.939, 255.0 - 103.939)
        init_image.assign(clipped)

        if loss < best_loss:
            best_loss = loss
            best_img = init_image.numpy()

        if i % 100 == 0:
            print(f"Iteration {i}, Total loss: {loss:.4f}")

    result = deprocess_image(best_img)
    cv2.imwrite(output_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
    print(f"Saved styled image to {output_path}")


In [6]:
# ✅ STEP 5: Run Style Transfer for 5 Content Images

# Style image name (uploaded earlier)
style_img = 'style.jpg'

# List of content image names (uploaded earlier)
content_images = [
    'content1.jpg',
    'content2.jpg',
    'content3.jpg',
    'content4.jpg',
    'content5.jpg'
]

os.makedirs("results", exist_ok=True)

for i, content_path in enumerate(content_images):
    output_path = f"results/stylized_{i+1}.jpg"
    run_style_transfer(content_path, style_img, output_path, iterations=2)


Iteration 0, Total loss: 1115315584.0000
Saved styled image to results/stylized_1.jpg
Iteration 0, Total loss: 2193614848.0000
Saved styled image to results/stylized_2.jpg
Iteration 0, Total loss: 1118142464.0000
Saved styled image to results/stylized_3.jpg


Iteration 0, Total loss: 1673658112.0000
Saved styled image to results/stylized_4.jpg


Iteration 0, Total loss: 1049971008.0000
Saved styled image to results/stylized_5.jpg


In [7]:
# ✅ STEP 6: Download Results (optional)
!zip -r stylized_results.zip results/
from google.colab import files
files.download("stylized_results.zip")


  adding: results/ (stored 0%)
  adding: results/stylized_3.jpg (deflated 1%)
  adding: results/stylized_1.jpg (deflated 0%)
  adding: results/stylized_2.jpg (deflated 1%)
  adding: results/stylized_4.jpg (deflated 2%)
  adding: results/stylized_5.jpg (deflated 1%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>